# Получение предсказаний обученной модели



## Библиотеки

In [1]:
# установить библиотеки в случае их отсутствия
# pip install pandas
# pip install numpy
# pip install scikit-learn
# pip install PyYAML 
import pandas as pd
import numpy as np
import json
import yaml

from sklearn.metrics import f1_score

from bert_inference import run_inference
from bert_model import BertForSequenceClassification


d:\prodocs\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open("config.yml", "r") as yamlfile:
    cfg = yaml.safe_load(yamlfile)

bert_model = BertForSequenceClassification(
        pretrained_model_name='DeepPavlov/rubert-base-cased',
        num_labels=cfg['model']['num_classes'],
        dropout=cfg['model']['dropout'],
        inference=True
        )


/home/don/DataScience/.venv_ds/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializi

## Предсказание для 1 документа

In [2]:
with open('labels.json', 'r', encoding='utf8') as f:
    kind_names = json.load(f)
kind_names
kind_names_rev = {v : int(i) for i, v in kind_names.items()}
kind_names_rev = {v : int(i) for i, v in kind_names.items()}



In [3]:
df = pd.read_csv('sample.csv')
doc = df.loc[4]['text']
doc


'Соглашение\nо расторжении договора об оказании услуг\nот "13" августа 2020 года № 23\n\n\nг. Сарапул\n"15" ноября 2020 года\n\nОбщество с ограниченной ответственностью "Сириус", далее именуемое "Заказчик", в лице генерального директора Грушева Сергея Сергеевича, действующего на основании Устава, с одной стороны, и общество с ограниченной ответственностью "Альдебаран", далее именуемое "Исполнитель", в лице Константинова Петра Петровича, действующего на основании Устава, с другой стороны, вместе именуемые "Стороны", а индивидуально - "Сторона", составили настоящее соглашение (далее - Соглашение) о нижеследующем:\n\n1.        По согласию обеих Сторон принято решение расторгнуть договор об оказании консультационных услуг от "13" августа 2020 года № 23 (далее - Договор).\n2.        Стороны произвели сверку расчетов по Договору (документ прилагается).\n3.        Заказчик в течение 5 (пяти) рабочих дней с момента подписания Соглашения обязуется перечислить Исполнителю сумму в размере 30 000 

In [9]:
most_confident, most_labels, getting_args_, probab= run_inference(doc, 'cpu', bert_model)


Inference:   0%|          | 0/1 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/don/DataScience/.venv_ds/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2619: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Inference: 100%|██████████| 1/1

In [10]:
print('Сходится класс документа -', kind_names[str(most_confident)] == df.loc[4]['class'])


Сходится класс документа - True


## Предсказание для списка документов


In [14]:
df['label'] = df['class'].map(kind_names_rev)


In [15]:
df_list = df.sample(50)
df_list


,class,text,label
180,contract,Договор подряда на уборку снега с крыш домов\n...,3
191,contract,Договор купли-продажи микшерного пульта\r\nг. ...,3
356,order,ООО «Кукушкин Двор»\r\n\r\n\r\nПРИКАЗ № 2\r\nо...,5
281,act,Акт распределения имущества при ликвидации ООО...,4
171,proxy,Д О В Е Р Е Н Н О С Т Ь\nг. Марс\nПервого янва...,2
338,order,"Общество с ограниченной ответственностью ""Мерк...",5
469,bill,\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t...,10
210,contract,Договор подряда на плановый ремонт системы авт...,3
222,contract,Договор подряда на техническое обслуживание си...,3
489,bill,\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t...,10


In [16]:
most_confident_label = list()
most_confident_labels = list()
getting_confidences_args = list()
probabilities = list()


In [17]:
%%time
for i in df_list['text'].values:
    most_confident, most_labels, getting_args_, probab= run_inference(i, 'cpu', bert_model)

    most_confident_label.append(most_confident)
    most_confident_labels.append(most_labels)
    getting_confidences_args.append(getting_args_)
    probabilities.append(probab)


Inference:   0%|          | 0/1 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/don/DataScience/.venv_ds/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2619: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Inference: 100%|██████████| 1/1

CPU times: user 3min 23s, sys: 2.87 s, total: 3min 26s
Wall time: 45.2 s


In [18]:
  df_list['predict'] = most_confident_label
  df_list


,class,text,label,predict
180,contract,Договор подряда на уборку снега с крыш домов\n...,3,3
191,contract,Договор купли-продажи микшерного пульта\r\nг. ...,3,3
356,order,ООО «Кукушкин Двор»\r\n\r\n\r\nПРИКАЗ № 2\r\nо...,5,5
281,act,Акт распределения имущества при ликвидации ООО...,4,4
171,proxy,Д О В Е Р Е Н Н О С Т Ь\nг. Марс\nПервого янва...,2,2
338,order,"Общество с ограниченной ответственностью ""Мерк...",5,5
469,bill,\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t...,10,10
210,contract,Договор подряда на плановый ремонт системы авт...,3,3
222,contract,Договор подряда на техническое обслуживание си...,3,3
489,bill,\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t...,10,10


In [22]:
print('Метрика f1 -', f1_score(df_list['label'], df_list['predict'], average='weighted'))


Метрика f1 - 0.9610582010582011
